# 如何修改/优化梯度计算

华东师范大学 数学科学学院 陈久宁 2019年12月11日

由于自动微分框架的存在，绝大部分时候我们并不需要再亲自去实现backward算法，
但是依然存在一些必须去手写梯度的情况，比如说：

* 数值精度 -- $\ln(e^x)$在x极大的情况下会因为$e^x$会导致数值溢出
* 计算效率 -- 自动微分框架会通过链式法则来计算导数，这对于有些可以约掉的计算项来说，会带来额外的计算开销

Reference: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd

In [ ]:
import torch

from sklearn import datasets
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_boston(ratio=0.8):
    X, Y = datasets.load_boston(True)
    Y.shape = -1, 1
    
    # normalization
    X = X/80
    Y = Y/(np.max(Y) - np.min(Y))
    
    num_samples = len(Y)
    num_train = math.ceil(num_samples * ratio)
    
    # 随机打乱数据
    idx = np.random.permutation(np.arange(num_samples))
    traindata = X[idx[:num_train]], Y[idx[:num_train]]
    validdata = X[idx[num_train:]], Y[idx[num_train:]]
    
    return traindata, validdata

自定义自动微分函数

In [ ]:
class SigmoidFunction(torch.autograd.Function):
    @staticmethod # 静态方法
    def forward(ctx, input): # context
        ctx.cache = torch.exp(-input)
        return 1/(1+ctx.cache)
    
    @staticmethod
    def backward(ctx, grad_output):
        cache = ctx.cache # 用前向传播的中间结果，避免重复计算
        dLdX = grad_output * (ctx.cache/torch.pow(ctx.cache+1, 2))

In [ ]:
class Sigmoid(torch.nn.Module):
    def forward(self, X):
        return SigmoidFunction.apply(X)

测试

In [ ]:
(X_train, Y_train), (X_valid, Y_valid) = load_boston()
X_train, Y_train = torch.Tensor(X_train), torch.Tensor(Y_train)
X_valid, Y_valid = torch.Tensor(X_valid), torch.Tensor(Y_valid)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(13, 50),
    Sigmoid(),
    torch.nn.Linear(50, 1)
)

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
valid_losses = []
train_losses = []
for i in range(1000):
    X, Y = X_train, Y_train
    
    opt.zero_grad()
    l = loss(model(X), Y)
    l.backward()
    opt.step()
    
    with torch.no_grad():
        cur_train_loss = l
        cur_valid_loss = loss(Y_valid, model(X_valid))
        valid_losses.append(cur_valid_loss) 
        train_losses.append(cur_train_loss)

        if i%100 == 0:
            print(f"Iter {i}: loss {cur_train_loss:.4f}, valid loss {cur_valid_loss:.4f}")

In [ ]:
plt.plot(train_losses)
plt.plot(valid_losses)
plt.legend(["train loss", "validation loss"])